In [1]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '1'

import logging
import numexpr as ne
import numpy as np
import torch
import datetime
from ddopai.envs.pricing.dynamic import DynamicPricingEnv
from ddopai.envs.pricing.dynamic_inventory import DynamicPricingInvEnv
from ddopai.envs.actionprocessors import ClipAction, RoundAction

from ddopai.experiments.experiment_functions_online import run_experiment
from ddopai.experiments.meta_experiment_functions import *
import requests
import yaml
import re
import pandas as pd
import wandb
from copy import deepcopy
import warnings
import gc
from mushroom_rl import core 
import pickle
from tqdm import tqdm, trange

In [ ]:
logging_level = logging.INFO
logging.basicConfig(level=logging_level)

ne.set_num_threads(1)
torch.backends.cudnn.enabled = False
torch.set_num_threads(1)

set_warnings(logging.INFO) # turn off warnings for any level higher or equal to the input level

project_name = "CMDP-Bandit"
config_sweep = import_config("config_sweep.yaml")
config_env = import_config("config_env.yaml")

INFO:root:Configuration file 'config_sweep.yaml' successfully loaded.
INFO:root:Configuration file 'config_env.yaml' successfully loaded.


In [3]:
artifacts = []
run = wandb.init(
    project=project_name,
    name = f"{project_name}_artifact_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
)
for artifact_index in trange(config_sweep["num_trials"]):
    
    raw_data, val_index_start, test_index_start = get_online_data(
            config_env,
            overwrite=False
        )
    
    with open('data/raw_data.pkl', 'wb') as f:
        pickle.dump(raw_data, f)
    artifact = wandb.Artifact('raw_data', type='data')
    artifact.add_file('data/raw_data.pkl')
    wandb.log_artifact(artifact)
    artifact.wait()
    artifacts.append(artifact.name)
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: timlachner. Use `wandb login --relogin` to force relogin


100%|██████████| 30/30 [02:29<00:00,  4.99s/it]


In [4]:
config_sweep["parameters"]["artifact"] = {"values": artifacts}


In [5]:
del config_sweep["num_trials"]
config_sweep

{'method': 'grid',
 'name': 'config_sweep',
 'parameters': {'config_train-agent': {'values': ['Clairvoyant',
    'SAC',
    'Greedy',
    'ILQX',
    'TS']},
  'artifact': {'values': ['raw_data:v0',
    'raw_data:v1',
    'raw_data:v2',
    'raw_data:v3',
    'raw_data:v4',
    'raw_data:v5',
    'raw_data:v6',
    'raw_data:v7',
    'raw_data:v8',
    'raw_data:v9',
    'raw_data:v10',
    'raw_data:v11',
    'raw_data:v12',
    'raw_data:v13',
    'raw_data:v14',
    'raw_data:v15',
    'raw_data:v16',
    'raw_data:v17',
    'raw_data:v18',
    'raw_data:v19',
    'raw_data:v20',
    'raw_data:v21',
    'raw_data:v22',
    'raw_data:v23',
    'raw_data:v24',
    'raw_data:v25',
    'raw_data:v26',
    'raw_data:v27',
    'raw_data:v28',
    'raw_data:v29']}}}

In [6]:
sweep_id = wandb.sweep(config_sweep, project=project_name)

Create sweep with ID: aelzufgr
Sweep URL: https://wandb.ai/timlachner/CMDP-Bandit/sweeps/aelzufgr


In [7]:
wandb.finish()